In [1]:
!pip install --upgrade openai -q
!pip install --upgrade boto3 -q
!pip install langchain -q
!pip install langchain-openai -q
!pip install langchain-community -q
!pip install pypdf -q
!pip install chromadb -q 
!pip install pandas -q
!pip install transformers 
!pip install -U sentence-transformers
!pip install kor
print('Done')


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.3 which is incompatible.


     ---------------------------------------- 0.0/14.7 MB ? eta -:--:--
     --------------------------------------- 0.0/14.7 MB 682.7 kB/s eta 0:00:22
     --------------------------------------- 0.1/14.7 MB 525.1 kB/s eta 0:00:28
      --------------------------------------- 0.3/14.7 MB 2.4 MB/s eta 0:00:07
     -- ------------------------------------- 1.0/14.7 MB 6.0 MB/s eta 0:00:03
     -- ------------------------------------- 1.0/14.7 MB 6.0 MB/s eta 0:00:03
     ---- ----------------------------------- 1.6/14.7 MB 6.0 MB/s eta 0:00:03
     ----- ---------------------------------- 2.1/14.7 MB 7.4 MB/s eta 0:00:02
     ----- ---------------------------------- 2.1/14.7 MB 7.4 MB/s eta 0:00:02
     ------- -------------------------------- 2.8/14.7 MB 6.7 MB/s eta 0:00:02
     -------- ------------------------------- 3.1/14.7 MB 7.4 MB/s eta 0:00:02
     --------- ------------------------------ 3.5/14.7 MB 6.9 MB/s eta 0:00:02
     ------------ --------------------------- 4.8/14.7 MB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.22 requires numpy>=1.22.5, but you have numpy 1.22.4 which is incompatible.


In [2]:
# Judge Using LLM Judge
# Use Claude + Open AI
# Use RAG to get treatment info
# Use COT + Few shot
# Use Automated prompt engineering, make llm create a prompt
# Self Consistency, have multiple outputs polled

In [7]:
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import random
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import pandas as pd


c:\Users\Balaj\anaconda3\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
medical_record_paths = [
    'task\medical-record-1.pdf',
    'task\medical-record-2.pdf',
    'task\medical-record-3.pdf'
]
record_index = 0

In [9]:
loader = PyPDFLoader(medical_record_paths[record_index])
pages = loader.load_and_split()
medical_record = '\n'.join([page.page_content for page in pages]) 
medical_record

'Co:Helm  \nMEDICAL RECORD Patient Name: James Freeman DOB: 06/16/1982 MRN: 456789123 Sex: Male  PRESENTING COMPLAINT Symptoms: Occasional rectal bleeding and abdominal discomfort for the past 6 months. Duration: 6 months.  PATIENT INFORMATION Name: James Freeman DOB: 06/16/1982 Gender: Male Address: 4521 Maple Avenue, Dallas, Texas 75219 Contact Number: (214) 555-0123 Emergency Contact: Not provided  MEDICAL HISTORY • Family History: Father had colorectal cancer at age 68. • Personal Medical History: Hypertension, managed with medication. • Medications: Lisinopril 10mg daily. • Allergies: No known drug allergies.  ALLERGIES • Allergies not reviewed (last reviewed 11/28/2022) • NKDA\nCo:Helm  \nVITALS Ht: 6ft2 Wt: 165lbs BMI: 31.2 Pulse: 96bpm  PHYSICAL EXAMINATION Findings: Mild tenderness in the lower abdomen, no palpable masses.  DIAGNOSTIC TESTS AND RESULTS Blood Tests: Hemoglobin slightly low at 12.4 g/dL, rest of the complete blood count within normal limits. Normal liver functio

### RAG to Fetch Relevent information

In [48]:
# Chunk the output

# Use the recursive character splitter
recur_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=60,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    is_separator_regex=True,
)

# Perform the splits using the splitter
data_splits = recur_splitter.split_documents(pages)
print(random.choice(data_splits).page_content)
print(len(data_splits))

PREVIOUS MEDICAL PROCEDURES: • Gallbladder Removal, Cholecystectomy (47562): Underwent a cholecystectomy several years ago. The postoperative course and its relation to current symptoms are unclear. (Laparoscopy, surgical; cholecystectomy). • Lower Back Pain Management: Received physical therapy for lower back pain and abdominal symptoms. Several weeks ago.
9


In [3]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
embeddings = model.encode([split.page_content for split in data_splits]).tolist()
len(embeddings)

12

In [4]:
### Using embeddings by MPNET
# model_name = "sentence-transformers/all-mpnet-base-v2"
model_name = "multi-qa-MiniLM-L6-cos-v1"
encode_kwargs = {"normalize_embeddings": False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, encode_kwargs=encode_kwargs
)

In [5]:
db = Chroma.from_documents(data_splits, hf_embeddings)

In [6]:
query = "What treatment has been recomended?"
# Retrieve similar chunks based on relevance. We only retrieve 'k' most similar chunks
similar_chunks = db.similarity_search_with_relevance_scores(query, k=4)

# Format document to text format
retrieved_text = [chunk[0].page_content for chunk in similar_chunks]
relevance_score = [chunk[1] for chunk in similar_chunks]

# Store and print as a dataframe
retrieved_chunks = pd.DataFrame(
    list(zip(retrieved_text, relevance_score)),
    columns=["Retrieved Chunks", "Relevance Score"],
)
with pd.option_context("display.max_colwidth", None):
    display(retrieved_chunks)

c:\Users\Balaj\anaconda3\envs\genai\lib\site-packages\langchain_core\vectorstores.py:311: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='NOTES Px underwent treatment for abdominal discomfort 1y ago. Rectal bleeding. Tx failed to yield significant improvement. Rpt 6wk ago led to noticeable improvement in symptoms.  CLINICAL IMPRESSION Possible internal hemorrhoids or polyps; colorectal cancer to be ruled out.', metadata={'page': 1, 'source': 'task\\medical-record-1.pdf'}), 0.10306870215481312), (Document(page_content='• Medications: Lisinopril 10mg daily. • Allergies: No known drug allergies.  ALLERGIES • Allergies not reviewed (last reviewed 11/28/2022) • NKDA', metadata={'page': 0, 'source': 'task\\medical-record-1.pdf'}), 0.07417670032580714), (Document(page_content='No solid foods, milk, or orange/red colored liquids. Regular medications can be taken with small sips of water. 3. Post-Procedure Care: Patient to be observed for any signs of complic

,Retrieved Chunks,Relevance Score
0,NOTES Px underwent treatment for abdominal discomfort 1y ago. Rectal bleeding. Tx failed to yield significant improvement. Rpt 6wk ago led to noticeable improvement in symptoms. CLINICAL IMPRESSION Possible internal hemorrhoids or polyps; colorectal cancer to be ruled out.,0.103069
1,• Medications: Lisinopril 10mg daily. • Allergies: No known drug allergies. ALLERGIES • Allergies not reviewed (last reviewed 11/28/2022) • NKDA,0.074177
2,"No solid foods, milk, or orange/red colored liquids. Regular medications can be taken with small sips of water. 3. Post-Procedure Care: Patient to be observed for any signs of complications such as severe pain, fever, or excessive bleeding. Advised to rest for the remainder of the day. Normal diet can be resumed the following day unless otherwise advised. 4.",0.002231
3,"Imaging Studies: None conducted. HISTORY Appendectomy (44970): Patient underwent an appendectomy due to acute appendicitis in 2015. (Laparoscopy, surgical, appendectomy). Knee Arthroscopy (29881): Underwent left knee arthroscopy due to meniscal tear in 2018. (Arthroscopy, knee, surgical; with meniscectomy [medial OR lateral, including any meniscal shaving]).",-0.064475


### Processing through LLM

In [92]:
# from langchain.prompts.chat import (
#     ChatPromptTemplate,
#     HumanMessagePromptTemplate,
# )


# medical_prompt = "{text}"
# human_message_prompt = HumanMessagePromptTemplate.from_template(medical_prompt)
# chat_prompt = ChatPromptTemplate.from_messages(
#     [system_message_prompt, human_message_prompt]
# )

# get a chat completion from the formatted messages
# chat(chat_prompt.format_prompt(text="I love programming.").to_messages())

AIMessage(content="J'adore la programmation.")

In [10]:
from langchain_openai import ChatOpenAI
# Sanity Check
gpt = ChatOpenAI(model='gpt-3.5-turbo')
gpt.invoke('You are a medical assistant and an expert in medical records. You are skilled in understanding, processing and extracting information from complex medical documents.')

AIMessage(content="As a medical assistant and an expert in medical records, I possess a range of skills and knowledge that enable me to effectively handle complex medical documents. Here are some key areas in which I excel:\n\n1. Medical Terminology: I have a deep understanding of medical terminology, including anatomy, physiology, and various medical conditions. This expertise allows me to comprehend and accurately interpret medical records, including diagnoses, procedures, and treatment plans.\n\n2. Knowledge of Medical Coding Systems: I am well-versed in various coding systems such as ICD-10 (International Classification of Diseases) and CPT (Current Procedural Terminology). This knowledge helps me accurately assign appropriate codes to medical procedures and diagnoses, ensuring proper documentation and billing.\n\n3. Attention to Detail: I possess a keen eye for detail, which is essential when working with complex medical records. I can identify inconsistencies, errors, or missing 

In [52]:
from openai import OpenAI
from IPython.display import Markdown
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

client = OpenAI()
gpt = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
system_prompt = "You are a medical assistant and an expert in medical records. You are skilled in understanding, processing and breaking down complex medical documents."

def call_model(prompt:str) -> str:
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    )
    return completion.choices[0].message.content


def call_model_lc(prompt:str) -> str:
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=prompt
        ),
    ]
    return gpt(messages).content
    

In [54]:
call_model_lc('what is what')

"I'm sorry, but I'm not sure what you are asking. Could you please provide more context or clarify your question?"

### Phase 0 - Extract Recomended Treatment

In [38]:
recomended_treatment_prompt = f"""
You are given a patient's health record below. It contains various sections on medical and clinical information on to the patient

{medical_record}

Use the health record to determine the treatment(s) recomended by the doctor and extract any CPT code(s) if present. 
"""

In [39]:
treatment_output = call_model(recomended_treatment_prompt)
Markdown(treatment_output)

Based on the patient's health record, the treatments recommended by the doctor are as follows:

1. Colonoscopy: Scheduled for 12/15/2023. The purpose is to investigate the cause of symptoms and to screen for colorectal cancer or polyps. This code represents a standard diagnostic colonoscopy.

The CPT code for this procedure is 45378.

Please note that there may be additional treatments or procedures not mentioned in the health record provided.

#### Phase 1 - Determine Past Treatment Success

In [79]:
past_treatment_prompt = f"""
You are given a patient's health record below. It contains various sections on medical and clinical information on to the patient

{medical_record}


Analyse the record. Take the following steps in order:
1. Identify the patient's current symptoms
2. Use the health record to identify any prior treatments if present.
3. Determine if the prior treatments improved the patient's current symptoms. 


Use your analysis to create the output below:

prior_treatments_success: Determine if the prior treatments improved the patient's current symptoms. (boolean)
evidence: Present evidence from the medical record for your answer to "prior_treatment_failed"
"""

In [80]:
"""
Your output must be in the json format described below:


[{{
"treatment": <Name of the treatment>
"success": <Boolean - True OR False>
"details": <the relevant details from the health record> 
}}]

Here is an example of an acceptable output:

[{{
"treatment": "Appendicectomy"
"successful": False
"details": "The patient underwent a appendicectomy few months ago. There were no further complications, the relation to current symptons is unclear"
""
}},
"""

'\nYour output must be in the json format described below:\n\n\n[{{\n"treatment": <Name of the treatment>\n"success": <Boolean - True OR False>\n"details": <the relevant details from the health record> \n}}]\n\nHere is an example of an acceptable output:\n\n[{{\n"treatment": "Appendicectomy"\n"successful": False\n"details": "The patient underwent a appendicectomy few months ago. There were no further complications, the relation to current symptons is unclear"\n""\n}},\n'

In [91]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | gpt | parser

output = chain.invoke({"query": past_treatment_prompt})

In [92]:
output

{'prior_treatments_success': False,
 'evidence': "The patient's current symptoms of persistent upper abdominal pain and heartburn have not improved with over-the-counter antacids started in June 2023."}

In [70]:
past_treatment_output = call_model(past_treatment_prompt)
Markdown(past_treatment_output)

Output:

1. Patient's Current Symptoms:
   - Occasional rectal bleeding
   - Abdominal discomfort for the past 6 months

2. Prior Treatments:
   The record mentions a prior treatment for abdominal discomfort 1 year ago, which failed to yield significant improvement.
   - No specific details about the treatment are mentioned.

3. Prior Treatments' Success:
   Based on the information provided in the record, it is unclear whether the prior treatments improved the patient's current symptoms. The record mentions that a repeat examination 6 weeks ago led to noticeable improvement in symptoms, but it does not specify the treatment given during that time. Therefore, it is not possible to determine if the prior treatments were successful in improving the patient's current symptoms.

Evidence:
   - The record does not provide specific evidence regarding the success or failure of the prior treatments. More detailed information about the treatments and their outcomes would be needed to draw any conclusions.

#### Phase X - Guideline Enrichment

In [84]:
colonoscopy_guidelines = """
[45378] Colonoscopy, flexible; diagnostic 
 
•  Colorectal cancer screening, as indicated by 1 or more of the following: 
o  Patient has average-risk or higher, as indicated by ALL of the following 
§  Age 45 years or older 
§  No colonoscopy in past 10 years 
o  High risk family history, as indicated by 1 or more of the following: 
§  Colorectal cancer diagnosed in one or more first-degree relatives of any age and ALL of the following: 
•  Age 40 years or older 
•  Symptomatic (eg, abdominal pain, iron deficiency anemia, rectal bleeding) 
§  Family member with colonic adenomatous polyposis of unknown etiology 
o  Juvenile polyposis syndrome diagnosis indicated by 1 or more of the following: 
§  Age 12 years or older and symptomatic (eg, abdominal pain, iron deficiency anemia, rectal bleeding, 
telangiectasia) 
§  Age younger than 12 years and symptomatic (eg, abdominal pain, iron deficiency anemia, rectal bleeding, 
telangiectasia) 
"""

In [55]:
enrichment_prompt = f""" You are given the following medical guidelines:

{colonoscopy_guidelines}

These guidelines describe criteria for a treatment. The guidelines will be used along with patients' health records to determine if a treatment is needed. 

Enrich these guidelines by adding a section that details potential evidences to look for in the patients records to make a (Yes or No) decision about the treatment.   
"""

In [89]:
enrichment_prompt_2 = f""" You are given the following medical guidelines:

{colonoscopy_guidelines}

These guidelines describe criteria for a treatment. The guidelines will be used along with patients' health records to determine if a treatment is needed. 

Use these guidelines to create sets of potential evidences to look for in the patients records that would qualify the patients for the treatment.   
"""

In [90]:
enrichment = call_model(enrichment_prompt_2) 
Markdown(enrichment)

Based on the provided medical guidelines for a colonoscopy, flexible (diagnostic), the following sets of potential evidences can be used to determine if a patient qualifies for the treatment:

1. Colorectal cancer screening criteria:
   - Age 45 years or older.
   - No colonoscopy in the past 10 years.

2. High-risk family history criteria:
   - Colorectal cancer diagnosed in one or more first-degree relatives of any age.
   - Age 40 years or older.
   - Symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding).

3. Family member with colonic adenomatous polyposis of unknown etiology.

4. Juvenile polyposis syndrome diagnosis criteria:
   - Age 12 years or older and symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia).
   - Age younger than 12 years and symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia).

These sets of potential evidences can be checked in the patient's medical records to determine if they meet the criteria for a colonoscopy, flexible (diagnostic) treatment.

In [85]:
naive_prompt = f""" You are given the following medical guidelines. These guidelines describe criteria for a treatment:

{colonoscopy_guidelines}

You are also given a patient's health record below.

{medical_record}

You must use the guidelines and patient's health record to determine if the patient qualifies for the treatment described in the guidelines.

For each point in the guidelines, determine if the patient qualifies or not based on their health record. 

In your output you must go through each point in the guidelines and do one of the following:
* Refer to qualifying evidence in the document if present.
* Mention evidence is missing to qualify.

At the end of your analysis, you should make a recomendation.
If qualifying evidence is found, the patient must be recomended for the treatment.
If qualifying evidence is not found, you must list all the additional peices of information that may be required to make the decision.

You must not assume or make up information about the patient.
"""

In [86]:
naive_output = call_model(naive_prompt)
Markdown(naive_output)

Based on the provided guidelines and the patient's health record, let's go through each point to determine if the patient qualifies for the treatment.

1. Colorectal cancer screening, as indicated by 1 or more of the following:
   - Patient has average-risk or higher, as indicated by ALL of the following:
     - Age 45 years or older.
     - No colonoscopy in the past 10 years.

Evidence in the health record: The patient had a colonoscopy on 07/22/2020, which was within the past 10 years. Therefore, the patient does not meet the criteria for colorectal cancer screening based on age and timing.

Additional information needed: None. The patient does not qualify based on the provided evidence.

2. High-risk family history, as indicated by 1 or more of the following:
   - Colorectal cancer diagnosed in one or more first-degree relatives of any age and ALL of the following:
     - Age 40 years or older.
     - Symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding).
   - Family member with colonic adenomatous polyposis of unknown etiology.

Evidence in the health record: There is no mention of a high-risk family history of colorectal cancer or colonic adenomatous polyposis in the patient's health record.

Additional information needed: Family history information is needed to determine if the patient qualifies based on high-risk family history.

3. Juvenile polyposis syndrome diagnosis indicated by 1 or more of the following:
   - Age 12 years or older and symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia).
   - Age younger than 12 years and symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia).

Evidence in the health record: There is no mention of a juvenile polyposis syndrome diagnosis or symptoms related to it in the patient's health record.

Additional information needed: No additional information is needed since there is no indication of juvenile polyposis syndrome in the health record.

Based on the provided evidence, the patient does not currently qualify for a colonoscopy, as indicated by the guidelines. The patient's last colonoscopy was within the past 10 years, and there is no evidence of high-risk family history or juvenile polyposis syndrome.

Please note that it is important to consult with the patient's healthcare provider for a comprehensive evaluation and to determine the most appropriate course of action based on the patient's specific health history and needs.

### Structured Data Extraction

In [11]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

# Schema Definition
schema = Object(
    id="health_record",
    description="Patient Health Record",
    attributes=[
        Object(
            id="patient_information",
            description="Patient Information",
            attributes=[
                Text(id="patient_name", description="Patient Name"),
                Text(id="dob", description="Date of Birth"),
                Text(id="mrn", description="Medical Record Number"),
                Text(id="sex", description="Gender"),
                Text(id="address", description="Address"),
                Text(id="contact_number", description="Contact Number"),
            ],
        ),
        Object(
            id="presenting_complaint",
            description="Presenting Complaint",
            attributes=[
                Text(id="symptoms", description="Symptoms"),
                Text(id="duration", description="Duration"),
            ],
        ),
        Object(
            id="medical_history",
            description="Medical History",
            attributes=[
                Text(id="family_history", description="Family History"),
                Object(
                    id="personal_medical_history",
                    description="Personal Medical History",
                    attributes=[
                        Object(id="Diagnosis", description="Past Patient Diagnosis",
                             attributes=[
                                Text(id="Name", description="Name of the Diagnosis"),
                                Text(id="Date", description="Date of the Diagnosis"),
                                Text(id="Details", description="Details of the Diagnosis"),
                             ]),
                        Object(id="Treatments", description="Past Treatments",
                            attributes=[
                                Text(id="Name", description="Name of the Treatment"),
                                Text(id="Date", description="Date of the Treatment"),
                                Text(id="Details", description="Details of the Treatment"),
                             ]),
                    ],
                ),
                Text(id="medications", description="Medications"),
                Text(id="allergies", description="Allergies"),
            ],
        ),
        Object(
            id="vitals",
            description="Vitals",
            attributes=[
                Number(id="height", description="Height"),
                Number(id="weight", description="Weight"),
                Number(id="bmi", description="BMI"),
                Number(id="pulse", description="Pulse"),
            ],
        ),
        Text(id="Diagnostic Tests", description="Diagnostic Tests"),
        Text(id="history", description="History"),
        Text(id="clinical_impression", description="Clinical Impression"),
        Text(id="laboratory_results", description="Laboratory Results"),
        Object(
            id="plan",
            description="Plan",
            attributes=[
                Text(id="planned treatments", description="Treatments planned"),
                Text(id="pre_procedure_instructions", description="Pre-Procedure Instructions"),
                Text(id="post_procedure_care", description="Post-Procedure Care"),
                Text(id="consent", description="Consent"),
            ],
        ),
        Text(id="follow_up", description="Follow-Up"),
    ],
)

# Langchain LLM Initialization (Assuming you have the llm and chain initialized as per the provided documentation)

# Create Extraction Chain
chain = create_extraction_chain(gpt, schema, encoder_or_encoder_class="json", input_formatter=None)

# Run extraction
result = chain.run((medical_record))

# Extracted data
extracted_data = result['data']['health_record']

# Output the extracted data (replace with your desired output format)
print(extracted_data)


c:\Users\Balaj\anaconda3\envs\genai\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'patient_information': {'patient_name': 'James Freeman', 'dob': '06/16/1982', 'mrn': '456789123', 'sex': 'Male', 'address': '4521 Maple Avenue, Dallas, Texas 75219', 'contact_number': '(214) 555-0123'}, 'presenting_complaint': {'symptoms': 'Occasional rectal bleeding and abdominal discomfort for the past 6 months.', 'duration': '6 months.'}, 'medical_history': {'family_history': 'Father had colorectal cancer at age 68.', 'personal_medical_history': {'Diagnosis': {'Name': 'Hypertension', 'Date': '', 'Details': 'managed with medication'}, 'Treatments': {'Name': 'Lisinopril 10mg daily', 'Date': '', 'Details': ''}}, 'medications': 'Lisinopril 10mg daily', 'allergies': 'No known drug allergies'}, 'vitals': {'height': 6.2, 'weight': 165, 'bmi': 31.2, 'pulse': 96}, 'Diagnostic Tests': 'Blood Tests: Hemoglobin slightly low at 12.4 g/dL, rest of the complete blood count within normal limits. Normal liver function tests. Imaging Studies: None conducted.', 'history': 'Appendectomy (44970): Patie

In [12]:
result

{'data': {'health_record': {'patient_information': {'patient_name': 'James Freeman',
    'dob': '06/16/1982',
    'mrn': '456789123',
    'sex': 'Male',
    'address': '4521 Maple Avenue, Dallas, Texas 75219',
    'contact_number': '(214) 555-0123'},
   'presenting_complaint': {'symptoms': 'Occasional rectal bleeding and abdominal discomfort for the past 6 months.',
    'duration': '6 months.'},
   'medical_history': {'family_history': 'Father had colorectal cancer at age 68.',
    'personal_medical_history': {'Diagnosis': {'Name': 'Hypertension',
      'Date': '',
      'Details': 'managed with medication'},
     'Treatments': {'Name': 'Lisinopril 10mg daily',
      'Date': '',
      'Details': ''}},
    'medications': 'Lisinopril 10mg daily',
    'allergies': 'No known drug allergies'},
   'vitals': {'height': 6.2, 'weight': 165, 'bmi': 31.2, 'pulse': 96},
   'Diagnostic Tests': 'Blood Tests: Hemoglobin slightly low at 12.4 g/dL, rest of the complete blood count within normal limits.

In [26]:
from IPython.display import JSON
JSON(extracted_data)

<IPython.core.display.JSON object>

In [ ]:
structured_enriched_guidelines = """
**Guidelines for Colonoscopy, Flexible; Diagnostic:**

1. **Colorectal Cancer Screening:**
   - *Evidence to Look For:*
     - Age verification (45 years or older).
     - Confirmation of no colonoscopy in the past 10 years.
   - *Relevant Sections:*
     - Patient's date of birth.
     - Documentation of previous colonoscopies in the medical history.
   - *Additional Information:*
     - Request details on any previous colonoscopies that might not be clearly documented.

2. **High-Risk Family History:**
   - *Evidence to Look For:*
     - Family history of colorectal cancer in first-degree relatives.
     - Age of affected family members (40 years or older).
     - Symptoms in affected family members (abdominal pain, iron deficiency anemia, rectal bleeding).
     - Family history of colonic adenomatous polyposis of unknown etiology.
   - *Relevant Sections:*
     - Family history section.
   - *Additional Information:*
     - Request specific details on the age, symptoms, and diagnosis of the patient's father with colorectal cancer.

3. **Juvenile Polyposis Syndrome Diagnosis:**
   - *Evidence to Look For:*
     - Age verification (12 years or older for symptomatic cases).
     - Symptoms (abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia).
   - *Relevant Sections:*
     - Patient's date of birth.
     - Presenting complaint section for symptoms.
   - *Additional Information:*
     - Request information about the patient's age at the onset of symptoms and details if symptoms are present.
"""

In [ ]:
example_analysis_additional_information = """
**Example of Analysis**

- **Colorectal Cancer Screening:**
  - Patient is 57 years old (satisfying age criteria).
  - No clear documentation of a colonoscopy in the past 10 years.
  - *Additional Information:*
    - Request clarification on any undisclosed colonoscopies.

- **High-Risk Family History:**
  - Father had colorectal cancer (meeting family history criteria).
  - Age at diagnosis is not specified.
  - Symptoms of the father are not fully described.
  - *Additional Information:*
    - Request details on the age and symptoms of Patients's father during colorectal cancer diagnosis.

- **Juvenile Polyposis Syndrome Diagnosis:**
  - No indication of juvenile polyposis syndrome in the medical record.
  - *Additional Information:*
    - Inquire about any history or symptoms related to juvenile polyposis syndrome.

"""

example_analysis_qualifies = """
**Example of Analysis**

- **Colorectal Cancer Screening:**
  - Patient is 40 years old.
  - No clear documentation of a colonoscopy in the past 10 years.
  - *Additional Information:*
    - Request clarification on any undisclosed colonoscopies.

- **High-Risk Family History:** (Qualifies)
  - Father had colorectal cancer (meeting family history criteria).
  - Father's age at diagnosis 55.
  - Symptoms of the father include: abdominal pain and rectal bleeding .
  - *Additional Information:*
    - No Additional Information Required

- **Juvenile Polyposis Syndrome Diagnosis:**
  - No indication of juvenile polyposis syndrome in the medical record.
  - *Additional Information:*
    - Inquire about any history or symptoms related to juvenile polyposis syndrome.

    
Patient eligible for treatment due to High-Risk Family History
"""

In [ ]:
enriched_guidelines_2 = """
Structured Guidelines for Colonoscopy, Flexible; Diagnostic:

1. **Colorectal Cancer Screening:**
   - Indicated for individuals with average-risk or higher, meeting ALL of the following criteria:
     - Age 45 years or older.
     - No colonoscopy in the past 10 years.

2. **High-Risk Family History:**
   - Indicated for individuals with a high-risk family history, meeting 1 or more of the following criteria:
     - Colorectal cancer diagnosed in one or more first-degree relatives of any age, meeting ALL of the following:
       - Age 40 years or older.
       - Symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding).
     - Family member with colonic adenomatous polyposis of unknown etiology.

3. **Juvenile Polyposis Syndrome Diagnosis:**
   - Indicated for individuals with a juvenile polyposis syndrome diagnosis, meeting 1 or more of the following criteria:
     - Age 12 years or older and symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia).
     - Age younger than 12 years and symptomatic (e.g., abdominal pain, iron deficiency anemia, rectal bleeding, telangiectasia).
"""

### Using Claude

In [27]:
!pip install --upgrade --quiet  boto3

In [46]:
import boto3

In [ ]:
bedrock = boto3.client(
    service_name="bedrock",
    region_name="us-east-1",
    endpoint_url="https://bedrock.us-east-1.amazonaws.com",
)
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [44]:
from langchain_community.llms import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4000,
    "temperature": 0.0,
    "top_p": 1,
}

claude = Bedrock(
    credentials_profile_name="default", model_id="anthropic.claude-v2:1", model_kwargs=inference_modifier
)

In [45]:
claude('whats the capital of spain')


' The capital of Spain is Madrid.'